In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.utils import compute_class_weight
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix

In [2]:
X_attacker = np.load(Path("ClosestCarCSVsModel2/np_attacker.npy")) #ClosestCarCSVsModel2/np_attacker.npy
X_normal = np.load(Path("ClosestCarCSVsModel2/np_normal.npy"))

X_normal_train, X_normal_test = train_test_split(X_normal, test_size=0.10, shuffle=True)
X_attacker_train, X_attacker_test = train_test_split(X_attacker, test_size=0.33, shuffle=True)

#If I want to even out the test set just chop some of the X_normal_test
X_test = np.concatenate((X_normal_test, X_attacker_test), axis=0)
y_test = np.asarray([0]*X_normal_test.shape[0] + [1]*X_attacker_test.shape[0]).astype('float32')
print(X_attacker_train.shape)
print(y_test.shape)

(6420, 100, 12)
(6959,)


In [ ]:
np.sum(y_test) / y_test.shape

In [ ]:
def get_data(X_attacker, X_normal):
    X_normal_split, p = train_test_split(X_normal, test_size=0.80, shuffle=True)
    y = np.asarray([0]*X_normal_split.shape[0] + [1]*X_attacker.shape[0]).astype('float32')
    X = np.concatenate((X_normal_split, X_attacker), axis=0)
    return X, y

In [ ]:
X_train, y_train = get_data(X_attacker_train, X_normal_train)
np.sum(y_train)/len(y_train)

# CNN

In [ ]:
n_feats = 12
# X_train = np.expand_dims(X_train, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

N = 64
model = Sequential()
model.add(Conv2D(N, (3, 3), activation='relu', input_shape=(X_train.shape[1], n_feats, 1), padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*2, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*4, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*16, (3, 3), activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# model = resnet50(input_shape=(100, 12, 1), num_classes=2)

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


mc = ModelCheckpoint('best_model2_V3.h5', monitor='val_loss', mode='min', save_best_only=True)

for i in range(0,400):
    print('Epoch:', i)
    X_train, y_train = get_data(X_attacker_train, X_normal_train)
#     class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)
#     class_weights = dict(zip(np.unique(y_train), class_weights))
    
    model.fit(X_train, y_train, 
              epochs=1, 
              batch_size=30000, 
#               class_weight=class_weights,
#               shuffle=True,
#               validation_split=0.1,
#               callbacks=[mc],
              verbose = 0
              )

model.save('model2_V3.h5')
y_pred = model.predict(X_test, batch_size=9000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

In [4]:
model = keras.models.load_model('model2_V3.h5')
y_pred = model.predict(X_test[0:1])
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

1/1 [==============================] - 0s 62ms/step


ValueError: Found input variables with inconsistent numbers of samples: [6959, 1]

In [ ]:
# model.save('model2_V2_allscale.h5')
print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))
tn, fp, fn, tp = confusion_matrix(y_test, y_pred.flatten()).ravel()
print(f'TN:{tn}, FP:{fp}, FN:{fn}, TP:{tp}')

# LSTM

In [ ]:
n_feats = 16

model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(None, n_feats)))) # "None" allows for varying input lengths
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


mc = ModelCheckpoint('best_model2_V2.h5', monitor='val_loss', mode='min', save_best_only=True)

for i in range(0,100):
    print('Epoch:', i)
    X_train, y_train = get_data(X_attacker_train, X_normal_train)
    model.fit(X_train, y_train, 
              epochs=1, 
              batch_size=9000, 
              validation_split=0.1,
              callbacks=[mc])

model.save('model2_V2.h5')
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred.flatten()).ravel()
print(f'TN:{tn}, FP:{fp}, FN:{fn}, TP:{tp}')

# Transformer

https://keras.io/examples/nlp/text_classification_with_transformer/

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import TransformerBlock

# Prepare the data
input_shape = (100,12)  # input shape of each time series sequence
num_classes = 2  # number of classes for classification

# Define the model
inputs = Input(shape=input_shape)
x = inputs
embedding_dim = 12 
num_heads = 8
ff_dim = 12 

# Time series embedding layer
x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)

n_feats = 12


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


mc = ModelCheckpoint('best_model2_V2.h5', monitor='val_loss', mode='min', save_best_only=True)

# for i in range(0,100):
print('Epoch:', i)
X_train, y_train = get_data(X_attacker_train, X_normal_train)
y_train = tf.keras.utils.to_categorical(y_train)


model.fit(X_train, y_train, 
          epochs=10, 
          batch_size=1000, 
          validation_split=0.1,
          callbacks=[mc])

model.save('model2_V2.h5')
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))